In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

path = 'https://frenzy86.s3.eu-west-2.amazonaws.com/fav/tecno/titanic.csv'
df=pd.read_csv(filepath_or_buffer=path,sep='\t')

In [30]:
print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
151          152         1       1   
152          153         0       3   
153          154         0       3   
154          155         0       3   
155          156         0       1   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [31]:
#drop the non-required columns
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True,axis=1)

X=df.drop(columns='Survived',axis=1)
y=df['Survived']

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'], test_size=0.2, random_state=42)

In [38]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
56,2,female,21.0,0,0,10.5000,S
138,3,male,16.0,0,0,9.2167,S
123,2,female,32.5,0,0,13.0000,S
128,3,female,NaN,1,1,22.3583,C
127,3,male,24.0,0,0,7.1417,S
...,...,...,...,...,...,...,...
71,3,female,16.0,5,2,46.9000,S
106,3,female,21.0,0,0,7.6500,S
14,3,female,14.0,0,0,7.8542,S
92,1,male,46.0,1,0,61.1750,S


In [32]:
#1st Imputation Transformer
trf1 = ColumnTransformer([
        ('impute_age',SimpleImputer(),[2]),
        ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
    ],remainder='passthrough')

#2nd One Hot Encoding
trf2 = ColumnTransformer([
        ('ohe_sex_embarked', OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
    ], remainder='passthrough')

#3rd Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

#4th Feature selection
trf4 = SelectKBest(score_func=chi2,k=8)

#5th Model
trf5 = DecisionTreeClassifier()

In [33]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

# Display Pipeline
from sklearn import set_config
set_config(display='diagram')
#fit data
pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7a6a1d27a7a0>)),
                ('trf5', DecisionTreeClassifier())])

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

params = {'trf5__max_depth':[1,2,3,4,5,None] }

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

print(grid.best_score_)

0.637


In [35]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

params = {'trf5__max_depth':[1,2,3,4,5,None] }

grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X, y)

print(grid.best_score_)

0.6413306451612902


In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_test,pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.68      1.00      0.81        21
           1       1.00      0.09      0.17        11

    accuracy                           0.69        32
   macro avg       0.84      0.55      0.49        32
weighted avg       0.79      0.69      0.59        32



In [37]:
import joblib
joblib.dump(pipe, 'my_model_knn.pkl.pkl')


['my_model_knn.pkl.pkl']